## Requests test

In [9]:
from app.crud import annotation as crud_annotation
from app.core.db import get_mongodb_engine
from app.models import Annotation

In [16]:
engine = get_mongodb_engine()
annotations = await engine.find(Annotation)

In [17]:
annotations

[]

In [2]:
import base64
from pathlib import Path
import requests
file_path = Path("assets/2501.00663v1.pdf")
with open(file_path, "rb") as f:
    file_bytes = f.read()
encoded_content = base64.b64encode(file_bytes).decode("utf-8")
payload = {
    "name": "assets/2501.00663v1.pdf",
    "content": encoded_content
}
response = requests.post("http://localhost:8000/api/v1/document/upload", json=payload)

In [4]:
response.status_code

200

In [3]:
response.json()

{'id': 'e67bd4e4-0790-4913-a8be-7fe73d6e5c57',
 'name': 'assets/2501.00663v1.pdf',
 'path': 'e67bd4e4-0790-4913-a8be-7fe73d6e5c57.pdf',
 'metadata': None,
 'created_at': '2025-02-16T05:06:27.776000Z'}

In [ ]:
response

In [7]:
file_id = response.json()["id"]

In [8]:
payload = {
    "file_id": file_id
}
response = requests.post("http://localhost:8000/api/v1/document/process", json=payload)

In [9]:
response.json()

{'msg': 'Document processed successfully.'}

In [26]:
import requests
payload = {
    "file_id": file_id,
    "question": "What is the main result of this paper?",
    "k": 5
}
response = requests.post("http://localhost:8000/api/v1/document/rag", json=payload)

In [27]:
response.json()

{'status': 'success',
 'response': "There is no specific answer to this text as it appears to be a passage discussing recent advancements in linear recurrent models, their differences, and their connections to other architectures. The passage does not pose a question that requires a step-by-step solution or a final numerical answer.\n\nHowever, if you'd like me to summarize the main points of the passage, I can provide that:\n\nThe passage discusses various recent developments in linear recurrent models, including data-independent transition matrix/decay mechanisms, gating mechanisms, and more complex memory updating rules based on meta-learning, online learning, and/or delta-rule. It also mentions the connection between these models and other architectures, such as Transformers.\n\nIf you have a specific question related to linear recurrent models or their applications, I can try to help you with that.",
 'question': 'What is the main result of this paper?',
 'answer': 5,
 'section': 

In [30]:
response = requests.post("http://localhost:8000/api/v1/document/get", json={"file_id": file_id})

In [32]:
document, annotations, concepts = response.json()

## Access vector store

In [ ]:
import lancedb
from app.core.config import settings
from app.core.vector_store import init_vector_store
embeddings = settings.EMBEDDINGS_CLS()
lance_db_conn = lancedb.connect(settings.LANCE_URI)
lance_db = init_vector_store(lance_db_conn, embeddings, table_name=settings.LANCE_TABLE_NAME)
res = lance_db.similarity_search(
    "what is the main result of this paper?", 
    k=5, 
    filter={"metadata.file_id": "7ec10cb0-fe8a-4bf8-8e6c-409a75d8d03c"})

/Users/whikwon/miniconda3/envs/nexusnote/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/whikwon/.cache/huggingface/modules/transformers_modules/jinaai/jina-clip-implementation/51f02de9f2cf8afcd3bac4ce996859ba96f9f8e9/modeling_clip.py:137: UserWarning: Flash attention requires CUDA, disabling
  warnings.warn('Flash attention requires CUDA, disabling')
/Users/whikwon/.cache/huggingface/modules/transformers_modules/jinaai/jina-clip-implementation/51f02de9f2cf8afcd3bac4ce996859ba96f9f8e9/modeling_clip.py:172: UserWarning: xFormers requires CUDA, disabling
  warnings.warn('xFormers requires CUDA, disabling')
